# Setup

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    from_json, col, to_timestamp, year as spark_year,
    month as spark_month, dayofmonth, lit, concat_ws, lpad
)
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType,
    IntegerType, TimestampType
)

# Khởi tạo SparkSession với cấu hình cho Delta Lake
spark = (
    SparkSession.builder
    .appName("Databricks_BronzeSilver_Pipeline")  # Tên ứng dụng
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Kích hoạt Delta Lake extension
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Sử dụng Delta catalog
    .getOrCreate()
)

# Cấu hình địa chỉ máy chủ Kafka thông qua ngrok
kafka_bootstrap_servers = "0.tcp.ap.ngrok.io:12442"  # Địa chỉ Kafka dùng để stream dữ liệu từ Producer
# Ví dụ địa chỉ ngrok thật: kafka_bootstrap_servers = "0.tcp.ap.ngrok.io:10802"

# Cảnh báo nếu chưa cập nhật giá trị ngrok
if kafka_bootstrap_servers == "YOUR_NGROK_HOSTNAME:YOUR_NGROK_PORT":
    print("!!! CẢNH BÁO: Vui lòng cập nhật `kafka_bootstrap_servers` với endpoint ngrok của bạn !!!")

# Tên Kafka topic mà dữ liệu thời tiết sẽ được gửi vào
kafka_topic = "test"

# Định nghĩa schema cho dữ liệu thời tiết được gửi từ Producer (Kafka)
# Schema này cũng sẽ được sử dụng ở tầng Silver để giữ chuẩn cấu trúc dữ liệu
weather_data_schema_from_producer = StructType([
    StructField("time", StringType(), True),  # Thời gian đo
    StructField("month", IntegerType(), True),  # Tháng
    StructField("year", IntegerType(), True),  # Năm
    StructField("temperature", DoubleType(), True),  # Nhiệt độ (°C)
    StructField("feelslike", DoubleType(), True),  # Cảm giác như (°C)
    StructField("wind", DoubleType(), True),  # Tốc độ gió (km/h)
    StructField("direction", StringType(), True),  # Hướng gió
    StructField("gust", DoubleType(), True),  # Gió giật (km/h)
    StructField("cloud", IntegerType(), True),  # Mức độ mây (%)
    StructField("humidity", IntegerType(), True),  # Độ ẩm (%)
    StructField("precipitation", DoubleType(), True),  # Lượng mưa (mm)
    StructField("pressure", DoubleType(), True),  # Áp suất (hPa)
    StructField("weather", StringType(), True),  # Tình trạng thời tiết (nắng, mưa, v.v.)
    StructField("label", StringType(), True)  # Nhãn dữ liệu nếu dùng cho supervised learning
])

# Định nghĩa các đường dẫn trong hệ thống tệp của Databricks (DBFS)
BASE_DBFS_PATH = "dbfs:/user/thanhtai/delta_pipeline"

# Các đường dẫn cho từng tầng trong kiến trúc Bronze/Silver/Gold
BRONZE_API_DBFS_PATH = f"{BASE_DBFS_PATH}/bronze/from_api"  # Dữ liệu lấy từ API qua Kafka
BRONZE_CSV_DBFS_PATH = f"{BASE_DBFS_PATH}/bronze/from_csv"  # Dữ liệu CSV được tải thủ công
SILVER_MERGED_DBFS_PATH = f"{BASE_DBFS_PATH}/silver/merged_weather"  # Dữ liệu đã được xử lý và hợp nhất
GOLD_FEATURES_DBFS_PATH = f"{BASE_DBFS_PATH}/gold/weather_features"  # Đặc trưng (features) cho mô hình ML
GOLD_PREDICTIONS_DBFS_PATH = f"{BASE_DBFS_PATH}/gold/weather_predictions"  # Kết quả dự đoán của mô hình
MODEL_DBFS_PATH = f"{BASE_DBFS_PATH}/model/weather_rf_model"  # Vị trí lưu trữ mô hình ML
CHECKPOINT_API_DBFS_PATH = f"{BASE_DBFS_PATH}/_checkpoints/bronze_from_api_dbfs"  # Vị trí checkpoint cho Kafka streaming

# Danh sách các đường dẫn cần tạo trong DBFS
paths_to_create_dbfs = [
    BASE_DBFS_PATH, BRONZE_API_DBFS_PATH, BRONZE_CSV_DBFS_PATH,
    SILVER_MERGED_DBFS_PATH, GOLD_FEATURES_DBFS_PATH,
    GOLD_PREDICTIONS_DBFS_PATH, MODEL_DBFS_PATH, CHECKPOINT_API_DBFS_PATH
]

# Tạo các thư mục DBFS nếu chưa tồn tại
for path in paths_to_create_dbfs:
    try:
        dbutils.fs.mkdirs(path)  # Tạo thư mục
        print(f"Created DBFS path: {path}")
    except Exception as e:
        print(f"Could not create DBFS path {path}: {e}")
        pass  # Bỏ qua lỗi nếu thư mục đã tồn tại hoặc lỗi nhỏ khác

# In ra thông báo hoàn tất thiết lập
print("Setup complete. Các đường dẫn DBFS đã được chuẩn bị.")


Created DBFS path: dbfs:/user/thanhtai/delta_pipeline
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/bronze/from_api
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/bronze/from_csv
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/silver/merged_weather
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/gold/weather_features
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/gold/weather_predictions
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/model/weather_rf_model
Created DBFS path: dbfs:/user/thanhtai/delta_pipeline/_checkpoints/bronze_from_api_dbfs
Setup complete. Các đường dẫn DBFS đã được chuẩn bị.


# Kafka to Bronze API (Streaming)
Đọc dữ liệu thời tiết từ Kafka (được gửi bởi producer) và lưu dưới dạng bảng Delta thô vào lớp Bronze

In [0]:
# Đọc dữ liệu từ Kafka stream (real-time) với các cấu hình phù hợp
raw_kafka_df_dbfs = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers)  # Địa chỉ Kafka server
    .option("subscribe", kafka_topic)  # Tên Kafka topic cần đọc
    .option("startingOffsets", "latest")  # Chỉ lấy dữ liệu mới từ Kafka (bỏ qua dữ liệu cũ)
    .option("failOnDataLoss", "false")  # Nếu Kafka đã xoá dữ liệu cũ (retention), vẫn tiếp tục xử lý mà không báo lỗi
    .option("kafka.request.timeout.ms", "120000")  # Nếu Kafka phản hồi chậm, Spark sẽ chờ đến 120 giây trước khi timeout
    .option("kafka.session.timeout.ms", "60000")  # Kafka giữ session này hoạt động nếu không có tín hiệu mới trong 60 giây
    .load()
)

# Parse JSON từ Kafka message (cột value), ánh xạ theo schema đã biết từ producer
parsed_api_df_dbfs = (
    raw_kafka_df_dbfs
    .selectExpr("CAST(value AS STRING) as json_string")  # Chuyển cột `value` từ bytes sang string
    .withColumn("data", from_json(col("json_string"), weather_data_schema_from_producer))  # Parse JSON theo schema đã định nghĩa
    .select("data.*")  # Trích xuất tất cả các trường bên trong struct `data` thành các cột riêng biệt
)

# Dừng query cũ nếu đang chạy để tránh lỗi checkpoint bị xung đột khi ghi đè
for s in spark.streams.active:
    # Kiểm tra nếu checkpoint path nằm trong mô tả nguồn dữ liệu của query
    if CHECKPOINT_API_DBFS_PATH in s.lastProgress["sources"][0]["description"]:
        print(f"Stopping existing stream: {s.id}")
        try:
            s.stop()  # Dừng query đang chạy
            s.awaitTermination()  # Chờ dừng hoàn tất
            dbutils.fs.rm(CHECKPOINT_API_DBFS_PATH, recurse=True)  # Xóa checkpoint cũ để tránh lỗi xung đột
            print(f"Removed old checkpoint: {CHECKPOINT_API_DBFS_PATH}")
        except Exception as e:
            print(f"Error stopping stream or removing checkpoint: {e}")  # In lỗi nếu có

# Ghi dữ liệu stream xuống Bronze layer (Delta format) với checkpoint mới
api_bronze_query_dbfs = (
    parsed_api_df_dbfs.writeStream
    .trigger(processingTime="30 seconds")  # Trigger xử lý mỗi 30 giây
    .format("delta")  # Ghi dữ liệu theo định dạng Delta Lake
    .outputMode("append")  # Ghi thêm (append) dữ liệu vào bảng
    .option("checkpointLocation", CHECKPOINT_API_DBFS_PATH)  # Lưu checkpoint để Spark biết tiếp tục từ đâu khi restart
    .option("mergeSchema", "true")  # Cho phép tự động mở rộng schema nếu có thay đổi nhỏ
    .start(BRONZE_API_DBFS_PATH)  # Bắt đầu ghi dữ liệu stream vào đường dẫn Bronze
)

print(f"Streaming query {api_bronze_query_dbfs.id} to {BRONZE_API_DBFS_PATH} started.")


Streaming query 825aeef5-bc6d-426b-ad82-0450155b05d1 to dbfs:/user/thanhtai/delta_pipeline/bronze/from_api started.


# CSV to Bronze/from_csv (Batch)
Đọc dữ liệu thời tiết từ một file CSV đã được upload lên DBFS và lưu dưới dạng bảng Delta thô vào lớp Bronze

In [0]:
# Đảm bảo đã upload file này lên DBFS tại: dbfs:/FileStore/thanhtai_sample_data/weather_data_from_source.csv
csv_input_dbfs_path = "dbfs:/FileStore/thanhtai_sample_data/weather_data_from_source.csv"

try:
    # Kiểm tra sự tồn tại của file CSV trên DBFS
    dbutils.fs.ls(csv_input_dbfs_path)
    print(f"File CSV được tìm thấy tại: {csv_input_dbfs_path}")

    # Đọc dữ liệu từ file CSV với header và tự động suy luận schema
    df_csv_dbfs_raw = (
        spark.read.format("csv")
        .option("header", "true")  # Dòng đầu tiên là tên cột
        .option("inferSchema", "true")  # Spark sẽ tự đoán kiểu dữ liệu của các cột
        .load(csv_input_dbfs_path)
    )

    # In ra schema của DataFrame để kiểm tra kết quả suy luận schema
    print("Schema của dữ liệu CSV sau khi Spark suy luận:")
    df_csv_dbfs_raw.printSchema()

    # Hiển thị 5 dòng đầu tiên để xác minh dữ liệu
    display(df_csv_dbfs_raw.limit(5))

    # Ghi dữ liệu vào Bronze layer dưới định dạng Delta (overwrite nếu tồn tại)
    (
        df_csv_dbfs_raw.write.format("delta")
        .mode("overwrite")  # Ghi đè dữ liệu cũ nếu có
        .option("mergeSchema", "true")  # Cho phép tự động mở rộng schema nếu có cột mới
        .save(BRONZE_CSV_DBFS_PATH)  # Lưu vào đường dẫn Bronze từ CSV
    )

    print(f"Ghi dữ liệu CSV (với schema suy luận) vào Bronze (DBFS) thành công: {BRONZE_CSV_DBFS_PATH}")

    # Hiển thị 5 dòng đầu tiên từ dữ liệu đã ghi để xác nhận kết quả
    display(
        spark.read.format("delta")
        .load(BRONZE_CSV_DBFS_PATH)
        .limit(5)
    )

except Exception as e:
    # Xử lý lỗi: nếu file không tồn tại thì báo rõ lý do, ngược lại in ra lỗi cụ thể
    if "java.io.FileNotFoundException" in str(e) or "Path does not exist" in str(e):
        print(f"LỖI: File CSV không tìm thấy tại '{csv_input_dbfs_path}'. Vui lòng upload file lên DBFS trước.")
    else:
        print(f"Lỗi khi xử lý CSV to Bronze: {e}")


File CSV được tìm thấy tại: dbfs:/FileStore/thanhtai_sample_data/weather_data_from_source.csv
Schema của dữ liệu CSV sau khi Spark suy luận:
root
 |-- time: timestamp (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- temperature: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- direction: string (nullable = true)
 |-- gust: double (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- pressure: integer (nullable = true)
 |-- weather: string (nullable = true)
 |-- label: string (nullable = true)



time,month,year,temperature,feelslike,wind,direction,gust,cloud,humidity,precipitation,pressure,weather,label
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower


Ghi dữ liệu CSV (với schema suy luận) vào Bronze (DBFS) thành công: dbfs:/user/thanhtai/delta_pipeline/bronze/from_csv


time,month,year,temperature,feelslike,wind,direction,gust,cloud,humidity,precipitation,pressure,weather,label
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower
2025-05-27T20:35:00.000+0000,5,2025,25.3,28.6,3.6,NW,5.5,0,94,1.0,1007,Light rain shower,Light rain shower


# Bronze to Silver Merge (Batch) 
Đọc dữ liệu từ hai bảng Bronze (API và CSV), thống nhất schema, gộp dữ liệu, loại bỏ trung lặp và lưu vào lớp Silver

In [0]:
# Đọc dữ liệu đã lưu ở Bronze layer từ API (Kafka) - định dạng Delta
print(f"Đọc dữ liệu từ Bronze API Table (DBFS): {BRONZE_API_DBFS_PATH}")
df_api_raw_dbfs = spark.read.format("delta").load(BRONZE_API_DBFS_PATH)

# Đọc dữ liệu đã lưu ở Bronze layer từ CSV - định dạng Delta
print(f"Đọc dữ liệu từ Bronze CSV Table (DBFS, schema được suy luận): {BRONZE_CSV_DBFS_PATH}")
df_csv_raw_dbfs = spark.read.format("delta").load(BRONZE_CSV_DBFS_PATH)

# Hiển thị schema gốc để so sánh
print("\nSchema gốc của df_api_raw_dbfs (từ Kafka producer):")
df_api_raw_dbfs.printSchema()
print("Schema gốc của df_csv_raw_dbfs (từ CSV, suy luận):")
df_csv_raw_dbfs.printSchema()

# --- Thống nhất Schema ---
# Sử dụng schema chuẩn của producer làm schema mục tiêu cho Silver layer
target_silver_schema = weather_data_schema_from_producer

# Hàm căn chỉnh schema của DataFrame cho khớp với schema mục tiêu
def align_df_to_schema_dbfs(df, schema_target, df_name="DataFrame"):
    df_aligned = df
    existing_cols_lower = {c.lower(): c for c in df_aligned.columns}  # Tạo dict ánh xạ tên cột (case-insensitive)
    select_expressions = []

    for field in schema_target.fields:
        target_col_name = field.name
        target_col_type = field.dataType
        source_col_name_found = None

        # Tìm cột tương ứng (không phân biệt hoa thường)
        if target_col_name.lower() in existing_cols_lower:
            source_col_name_found = existing_cols_lower[target_col_name.lower()]

        if source_col_name_found:
            current_col_type = df_aligned.schema[source_col_name_found].dataType
            if current_col_type == target_col_type:
                # Nếu kiểu dữ liệu giống nhau → giữ nguyên
                select_expressions.append(col(source_col_name_found).alias(target_col_name))
            else:
                # Nếu khác kiểu → ép kiểu sang schema mục tiêu
                print(f"Thông báo ({df_name}): Ép kiểu cột '{source_col_name_found}' từ {current_col_type} sang {target_col_type} (đổi tên thành '{target_col_name}').")
                try:
                    select_expressions.append(col(source_col_name_found).cast(target_col_type).alias(target_col_name))
                except Exception as cast_error:
                    print(f"CẢNH BÁO ({df_name}): Không thể ép kiểu cột '{source_col_name_found}' sang {target_col_type}. Đặt thành NULL. Lỗi: {cast_error}")
                    select_expressions.append(lit(None).cast(target_col_type).alias(target_col_name))
        else:
            # Nếu thiếu cột → thêm cột với NULL
            print(f"Thông báo ({df_name}): Cột '{target_col_name}' không tìm thấy. Thêm cột này với giá trị NULL.")
            select_expressions.append(lit(None).cast(target_col_type).alias(target_col_name))
    
    return df_aligned.select(select_expressions)  # Trả về DataFrame đã chuẩn hóa schema

# --- Chuẩn hóa schema từng nguồn dữ liệu ---
print(f"\n--- Căn chỉnh df_api_raw_dbfs ---")
df_api_aligned_dbfs = align_df_to_schema_dbfs(df_api_raw_dbfs, target_silver_schema, "df_api_raw_dbfs")

print(f"\n--- Căn chỉnh df_csv_raw_dbfs ---")
df_csv_aligned_dbfs = align_df_to_schema_dbfs(df_csv_raw_dbfs, target_silver_schema, "df_csv_raw_dbfs")

# Kiểm tra schema sau khi chuẩn hóa
print("\nSchema của df_api_aligned_dbfs (sau khi chuẩn hóa):")
df_api_aligned_dbfs.printSchema()
print("Schema của df_csv_aligned_dbfs (sau khi chuẩn hóa):")
df_csv_aligned_dbfs.printSchema()

# Gộp 2 nguồn dữ liệu lại bằng unionByName (phải cùng schema, cùng tên cột)
df_merged_bronze_dbfs = df_api_aligned_dbfs.unionByName(
    df_csv_aligned_dbfs,
    allowMissingColumns=False  # Không cho phép thiếu cột — vì đã chuẩn hóa trước
)

# Thống kê số dòng trước và sau khi loại bỏ bản ghi trùng
print(f"Số dòng trước khi dropDuplicates: {df_merged_bronze_dbfs.count()}")
df_silver_transformed_dbfs = (
    df_merged_bronze_dbfs
    .dropDuplicates(["year", "month", "time", "weather", "temperature", "direction"])
)  # Loại bỏ trùng lặp theo các cột quan trọng
print(f"Số dòng sau khi dropDuplicates: {df_silver_transformed_dbfs.count()}")

# Kiểm tra schema final của Silver
print("\nSchema của Silver DataFrame cuối cùng (DBFS):")
df_silver_transformed_dbfs.printSchema()

# Ghi dữ liệu chuẩn hóa (Silver layer) ra định dạng Delta
(
    df_silver_transformed_dbfs.write.format("delta")
    .mode("overwrite")  # Ghi đè bảng nếu đã tồn tại
    .option("overwriteSchema", "true")  # Cho phép ghi đè cả schema
    .save(SILVER_MERGED_DBFS_PATH)
)

print(f"\nGhi dữ liệu vào Silver (DBFS) thành công: {SILVER_MERGED_DBFS_PATH}")

# Hiển thị vài dòng từ Silver để xác nhận
display(
    spark.read.format("delta")
    .load(SILVER_MERGED_DBFS_PATH)
)


Đọc dữ liệu từ Bronze API Table (DBFS): dbfs:/user/thanhtai/delta_pipeline/bronze/from_api
Đọc dữ liệu từ Bronze CSV Table (DBFS, schema được suy luận): dbfs:/user/thanhtai/delta_pipeline/bronze/from_csv

Schema gốc của df_api_raw_dbfs (từ Kafka producer):
root
 |-- time: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- temperature: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- direction: string (nullable = true)
 |-- gust: double (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- weather: string (nullable = true)
 |-- label: string (nullable = true)

Schema gốc của df_csv_raw_dbfs (từ CSV, suy luận):
root
 |-- time: timestamp (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- temperature: d

time,month,year,temperature,feelslike,wind,direction,gust,cloud,humidity,precipitation,pressure,weather,label
null,null,null,null,null,null,null,null,null,null,null,null,null,null
2025-05-27 11:01:00,5,2025,24.0,26.0,21.2,N,31.1,100,94,0.01,1012.0,Moderate rain,Moderate rain
2025-05-27 11:04:00,5,2025,24.0,26.0,21.2,N,31.1,100,94,0.01,1012.0,Moderate rain,Moderate rain
2025-05-27 11:07:00,5,2025,24.0,26.0,21.2,N,31.1,100,94,0.01,1012.0,Moderate rain,Moderate rain
2025-05-27 11:10:00,5,2025,24.0,26.0,21.2,N,31.1,100,94,0.01,1012.0,Moderate rain,Moderate rain
2025-05-27 11:13:00,5,2025,24.0,26.0,21.2,N,31.1,100,94,0.01,1012.0,Moderate rain,Moderate rain
2025-05-27 20:56:00,5,2025,24.0,25.5,16.6,NNE,26.5,0,69,0.0,1015.0,Overcast,Overcast
2025-05-27 20:59:00,5,2025,24.0,25.5,16.6,NNE,26.5,0,69,0.0,1015.0,Overcast,Overcast
2025-05-27 21:33:00,5,2025,24.1,25.6,16.6,N,26.4,0,69,0.0,1016.0,Cloudy,Cloudy
10:00,5,2025,24.1,25.9,5.0,N,7.2,100,83,0.0,1013.0,Overcast,Overcast
